## LangChain Expression Language (LCEL) — Quick Notes

• LCEL is the modern way to build LLM pipelines in LangChain  
• It replaces old Chain classes with a simpler, composable style  
• Everything in LCEL is a **Runnable** (can be invoked)

### `|` (Pipe Operator)
• Passes output of one step to the next  
• Example: Prompt → LLM → Parser

### RunnableSequence
• Executes steps one after another  
• Created using the `|` operator  
• Most common LCEL pattern

### RunnableParallel
• Runs multiple chains at the same time  
• Useful for summary + sentiment + keywords together

### RunnableMap
• Applies multiple transformations to the same input  
• Lightweight operations (not heavy LLM calls)

### Streaming
• Streams tokens as the LLM generates output  
• Enables real-time chat-like responses

### Retry
• Automatically retries failed LLM calls  
• Improves reliability in production

### Fallbacks
• Uses a backup chain if the main chain fails  
• Ensures system never breaks completely

### RunnableLambda 
It is a way to turn a normal Python function into a LangChain runnable component, so it can be used inside chains just like an LLM, prompt, or retriever.

Think of it as:

“Wrap my Python logic so LangChain can treat it like a step in a chain.”

In [21]:
# Imports + a simple LLM that needs NO API key
from langchain_core.language_models.fake import FakeListLLM
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import (
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
    RunnableGenerator,
)
from dotenv import load_dotenv

load_dotenv() 

False

In [23]:
fake_llm = FakeListLLM(responses=[
    "FAKE_LLM_RESPONSE: LangChain LCEL makes pipelines composable."
])
parser = StrOutputParser()

prompt = PromptTemplate.from_template("Answer in 1 line: {question}")

#### | operator + RunnableSequence (Prompt → LLM → Parser)

In [26]:
# RunnableSequence using pipe operator |
sequence_chain = prompt | fake_llm | parser

out = sequence_chain.invoke({"question": "What is LCEL?"})
print(out)

FAKE_LLM_RESPONSE: LangChain LCEL makes pipelines composable.


#### RunnableMap (multiple transformations on same input)
In LCEL, “map-style” is often done by returning a dict from a Runnable.

In [29]:
#RunnableMap-like behavior using RunnableLambda returning a dict
map_chain = RunnableLambda(lambda x: {
    "original": x,
    "upper": x.upper(),
    "length": len(x),
})

print(map_chain.invoke("langchain"))

{'original': 'langchain', 'upper': 'LANGCHAIN', 'length': 9}


#### RunnableParallel (run multiple branches in parallel)

In [32]:
# Parallel branches
summary_branch = prompt | fake_llm | parser
len_branch = RunnableLambda(lambda d: len(d["question"]))  # expects dict input
echo_branch = RunnableLambda(lambda d: f"ECHO: {d['question']}")

parallel_chain = RunnableParallel({
    "summary": summary_branch,
    "question_length": len_branch,
    "echo": echo_branch
})

result = parallel_chain.invoke({"question": "Explain LCEL simply"})
print(result)


{'summary': 'FAKE_LLM_RESPONSE: LangChain LCEL makes pipelines composable.', 'question_length': 19, 'echo': 'ECHO: Explain LCEL simply'}


#### Streaming responses

Easiest working solution (no API key): use a generator runnable that accepts an iterator and yields chunks

This pattern works with LCEL streaming correctly:

A function that uses yield:

1. Does NOT return everything at once
2. Returns values one by one
3. Pauses after each yield
4. Resumes from where it left off

In [64]:
from langchain_core.runnables import RunnableGenerator

def stream_transform(input_iter):
    # input_iter is an iterator of incoming chunks (not a plain string)
    for item in input_iter:
        # item will be the original input string here
        for w in str(item).split():
            yield w + " "

stream_chain = RunnableGenerator(stream_transform)

for chunk in stream_chain.stream("This is a streaming demo without any API key"):
    print(chunk, end="")
print()


This is a streaming demo without any API key 


#### Retry (automatic retry if something fails)

In [43]:
state = {"tries": 0}

def flaky_fn(x):
    state["tries"] += 1 
    if state["tries"] < 2:
        raise RuntimeError("Temporary failure, please retry!")
    return f"Success on try #{state['tries']}: {x}"

flaky_runnable = RunnableLambda(flaky_fn).with_retry(stop_after_attempt=3)

print(flaky_runnable.invoke("Hello Retry"))

Success on try #2: Hello Retry


#### Fallbacks (backup runnable if primary fails)

In [47]:
primary = RunnableLambda(lambda x: (_ for _ in ()).throw(RuntimeError("Primary failed!")))
backup = RunnableLambda(lambda x: f"Backup worked for input: {x}")

fallback_chain = primary.with_fallbacks([backup])

print(fallback_chain.invoke("Hello Fallback"))

Backup worked for input: Hello Fallback
